In [112]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler

from imblearn.over_sampling import SMOTE
from keras.models import Sequential
from keras.layers import Dense 



# 1. Traitement des données 

Importation du jeu de données et division en données classe et caractéristique

In [192]:
dt = pd.read_csv("/Users/mallorylecorre/Desktop/Dm_Ia/cars_data-class.csv")
print(dt.head())

X = dt.drop('Impact_Environnement', axis=1)
y = dt['Impact_Environnement']

   Taille_Auto   Poids_Auto  Acceleration  Annee_Fabrication    Kilometrage  \
0     3.790317  8869.946374      2.038688               2006  135542.067972   
1     3.375892  1126.174079      2.067411               2010   33710.551662   
2     4.844971  6100.019066      4.839676               2016  112977.279109   
3     2.316172  5771.509867      1.202574               2014   50139.727635   
4     2.618825  5676.666211      5.470806               2009  191188.750210   

   Nombre_Portes  Vitesse_Max  Prix_Assurance  Categorie_Voiture    Decote  \
0              5   195.289066      184.125577                  1  0.104588   
1              5   249.979656      858.935088                  3  0.346064   
2              5   235.763320      302.138838                  4  0.394582   
3              4   216.802020     1298.671966                  5  0.422048   
4              4   234.004505      702.626517                  2  0.279959   

   Impact_Environnement  
0                     1  
1   

Division du jeu de données en jeu test et jeu d'entrainement respectivement 20 % et 40 %. "sratify=y" permet de prendre le même nombre de données dans chaque classe pour être sur que toute les classes soient représentées dans le jeu d'entrainement.

In [193]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


On standardise les données pour mettre toute les variables à la même échelle.

In [194]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Distribution des données :

In [195]:
dt['Impact_Environnement'].value_counts()

1    537
3    385
2     42
0     36
Name: Impact_Environnement, dtype: int64

On peut voir que le jeu de données est déséquilibrée avec une sur-représentation de la classe 1.

On équilibre les données en sur-échantillonant avec SMOTE

In [196]:
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)


On vérifie que le rééquillibrage a bien fonctionné

In [197]:
print("Nouvelle distribution :")
print(pd.Series(y_train).value_counts())

Nouvelle distribution :
1    430
3    430
2    430
0    430
Name: Impact_Environnement, dtype: int64


Les données sont équilibrées

# Création du modèle 

Création du modèle avec trois couches. Le modèle prend en input le nombre de caractéristiques avec la fonction "X_train.shape[1]". Le nombre de couche et le nombre de neurones par couches a été determiner en faisant plusieurs essais. Les combinaisons donnant les meilleurs résultats ont été gardé. Les foncitons relu et sigmoid ont été testé mais c'est relu qui était la plus efficace.

In [198]:
model = Sequential()
model.add(Dense(60, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(4, activation='softmax'))



compilation du modèle en tuilisant une fonction de perte qui correspond à une classification multi-class

In [199]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


Entrainement du modèle 

In [200]:
model.fit(X_train, y_train, epochs=200, batch_size=40, validation_split=0.2)


Epoch 1/200
35/35 [==============================] - 0s 3ms/step - loss: 1.2896 - accuracy: 0.4477 - val_loss: 1.4786 - val_accuracy: 0.1860
Epoch 2/200
35/35 [==============================] - 0s 1ms/step - loss: 1.1025 - accuracy: 0.6366 - val_loss: 1.2871 - val_accuracy: 0.3256
Epoch 3/200
35/35 [==============================] - 0s 908us/step - loss: 0.9131 - accuracy: 0.6999 - val_loss: 1.0994 - val_accuracy: 0.4099
Epoch 4/200
35/35 [==============================] - 0s 922us/step - loss: 0.7669 - accuracy: 0.7398 - val_loss: 0.9179 - val_accuracy: 0.5581
Epoch 5/200
35/35 [==============================] - 0s 938us/step - loss: 0.6755 - accuracy: 0.7936 - val_loss: 0.8963 - val_accuracy: 0.5727
Epoch 6/200
35/35 [==============================] - 0s 940us/step - loss: 0.6112 - accuracy: 0.8227 - val_loss: 0.8252 - val_accuracy: 0.6279
Epoch 7/200
35/35 [==============================] - 0s 957us/step - loss: 0.5581 - accuracy: 0.8401 - val_loss: 0.7989 - val_accuracy: 0.6802
Epo

# Evaluation du modèle

In [201]:
y_pred_probabilities = model.predict(X_test)
y_pred = np.argmax(y_pred_probabilities, axis=1)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

7/7 [==============================] - 0s 481us/step
Accuracy: 0.815


on a une accuracy de 0.83

In [202]:
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       0.25      0.29      0.27         7
           1       0.90      0.85      0.88       107
           2       0.21      0.33      0.26         9
           3       0.87      0.87      0.87        77

    accuracy                           0.81       200
   macro avg       0.56      0.58      0.57       200
weighted avg       0.84      0.81      0.82       200



On peut voir que les classes qui sont sous-représentées dans le jeu de données de bases (la classe 2 et 0) sont celles avec les moins bon résultats.